### Word Group Prepare For Youtube Tool

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "German"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = German
Lang Id = 8
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/2-Word Group Prepare For Youtube Tool"

Path(path).mkdir(parents=True, exist_ok=True)

#### Read Data

In [6]:
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"

In [7]:
df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = df_28.head(700)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,ist sie aber so aber was ist mit mir,644,648,und dann ist sie aber so aber was ist mit mir,E5wrYo4PLM4,https://www.youtube.com/watch?v=E5wrYo4PLM4&t=...
1,und nicht auf das was du nicht,391,393,und nicht auf das was du nicht hast,D2EWOivrzz8,https://www.youtube.com/watch?v=D2EWOivrzz8&t=...
2,nicht dass das so was ist das,687,690,oh nee sag nicht dass das so was ist das,m9hZzWA0R3k,https://www.youtube.com/watch?v=m9hZzWA0R3k&t=...
3,das ist ein dass das nicht so,578,580,das ist ein dass das nicht so,XGhAlBQ3gOY,https://www.youtube.com/watch?v=XGhAlBQ3gOY&t=...
4,das ist ja nicht das was ich,548,551,das ist ja nicht das was ich,X0g4THI7vEs,https://www.youtube.com/watch?v=X0g4THI7vEs&t=...
...,...,...,...,...,...,...
302,ja aber ich,304,307,ja aber ich,Fhj19V34vzU,https://www.youtube.com/watch?v=Fhj19V34vzU&t=...
303,und mit den,436,438,und mit den,xNbTBGajGdQ,https://www.youtube.com/watch?v=xNbTBGajGdQ&t=...
304,dass er mir,3880,3884,dass er mir,PwbNezj_pyw,https://www.youtube.com/watch?v=PwbNezj_pyw&t=...
305,dass in den,602,608,dass in den,CSOH6GQhBQ4,https://www.youtube.com/watch?v=CSOH6GQhBQ4&t=...


In [8]:
df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = df_200.head(700)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,oder da würde er sagen da ich glaube es geht l...,1499,1505,oder da würde er sagen da ich glaube es geht l...,L22SNtHYLCw,https://www.youtube.com/watch?v=L22SNtHYLCw&t=...
1,sehen kann dann machen wir alles zu genau also...,2117,2126,sehen kann dann machen wir alles zu genau also...,8BgkmfGO8SI,https://www.youtube.com/watch?v=8BgkmfGO8SI&t=...
2,wie es ihnen gehen muss und was jetzt los ist ...,408,414,wie es ihnen gehen muss und was jetzt los ist ...,3sTLcfBgJh4,https://www.youtube.com/watch?v=3sTLcfBgJh4&t=...
3,ist etwas nicht so wirklich ist und dann mache...,2745,2754,ist etwas nicht so wirklich ist und dann mache...,HWzDSgiLVkI,https://www.youtube.com/watch?v=HWzDSgiLVkI&t=...
4,du denn jetzt machen oder sagen ja was können ...,2046,2051,du denn jetzt machen oder sagen ja was können ...,L22SNtHYLCw,https://www.youtube.com/watch?v=L22SNtHYLCw&t=...
...,...,...,...,...,...,...
695,ihn doch nur für sein geld,282,284,du liebst ihn doch nur für sein geld,yr9WrUFM4IE,https://www.youtube.com/watch?v=yr9WrUFM4IE&t=...
696,einen oder zwei leute raus,277,279,einen oder zwei leute raus,Ut64cNI2kvg,https://www.youtube.com/watch?v=Ut64cNI2kvg&t=...
697,wer hat schon zeit für den,607,610,wer hat schon zeit für den anderen,O5unu7USjv8,https://www.youtube.com/watch?v=O5unu7USjv8&t=...
698,nein wirklich oh mein gott,617,621,nein wirklich oh mein gott,YwTXhNYG4gw,https://www.youtube.com/watch?v=YwTXhNYG4gw&t=...


In [9]:
df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = df_1000.head(700)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,der noch geld haben will wahrscheinlich eben d...,878,884,der noch geld haben will wahrscheinlich eben d...,e0PyOh2KMQw,https://www.youtube.com/watch?v=e0PyOh2KMQw&t=...
1,aber vielleicht leute zu haben die da sind wo ...,815,824,aber vielleicht leute zu haben die da sind wo ...,e0PyOh2KMQw,https://www.youtube.com/watch?v=e0PyOh2KMQw&t=...
2,ich glaube sie wollte sagen hey du bist nur ei...,3278,3285,ich glaube sie wollte sagen hey du bist nur ei...,twfbyHqyITc,https://www.youtube.com/watch?v=twfbyHqyITc&t=...
3,so stelle ich mir das jetzt vor weil wenn man ...,1803,1810,so stelle ich mir das jetzt vor weil wenn man ...,URWnSf1sEDI,https://www.youtube.com/watch?v=URWnSf1sEDI&t=...
4,das hat damit zu tun dass wir einfach wissen d...,2804,2809,das hat damit zu tun dass wir einfach wissen d...,V1rKsO0okzY,https://www.youtube.com/watch?v=V1rKsO0okzY&t=...
...,...,...,...,...,...,...
695,werden weil wenn von uns da jemand sitzen würde,1025,1030,einbezogen werden weil wenn von uns da jemand ...,bXt7do8CjxU,https://www.youtube.com/watch?v=bXt7do8CjxU&t=...
696,am nächsten tag machte er sich auf den weg nach,1404,1407,am nächsten tag machte er sich auf den weg nac...,kLZyfnwuFAk,https://www.youtube.com/watch?v=kLZyfnwuFAk&t=...
697,am abend vorher meine sachen hier auf den boden,1608,1611,ich warf am abend vorher meine sachen hier auf...,XLxx_oJVkiM,https://www.youtube.com/watch?v=XLxx_oJVkiM&t=...
698,am nächsten tag machte er sich auf den weg nach,605,609,am nächsten tag machte er sich auf den weg nac...,1uzFP9Y9gZk,https://www.youtube.com/watch?v=1uzFP9Y9gZk&t=...


In [10]:
df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = df_5000.head(700)
df_5000

,search_string,start_time,end_time,sentence,video_id,video_url
0,lassen wir haben nachher sehr oft da drüber ge...,608,618,ja meine frau hat sich das nie so anmerken las...,rHKdeRoDJDs,https://www.youtube.com/watch?v=rHKdeRoDJDs&t=...
1,einige haben ja ganz schön viel erlebt ich den...,366,381,einige haben ja ganz schön viel erlebt ich den...,_W671aWBZL4,https://www.youtube.com/watch?v=_W671aWBZL4&t=...
2,davon glaube ich geht man so stück für stück w...,1727,1736,davon glaube ich geht man so stück für stück w...,kg23NfjKeTc,https://www.youtube.com/watch?v=kg23NfjKeTc&t=...
3,eine gewisse sicherheit bekommen dass das was ...,135,147,eine gewisse sicherheit bekommen dass das was ...,gweZAS7SiOU,https://www.youtube.com/watch?v=gweZAS7SiOU&t=...
4,in diese welt gut dann können wir uns vielleic...,1552,1560,in diese welt gut dann können wir uns vielleic...,BS0krBY3O8I,https://www.youtube.com/watch?v=BS0krBY3O8I&t=...
...,...,...,...,...,...,...
695,machen willst dein leben lang machen wolltest ...,318,322,machen willst dein leben lang machen wolltest ...,X2kuDag0LA4,https://www.youtube.com/watch?v=X2kuDag0LA4&t=...
696,wahrscheinlich wollte er sich von seinem alten...,3251,3256,wahrscheinlich wollte er sich von seinem alten...,SsNCFbkXGKE,https://www.youtube.com/watch?v=SsNCFbkXGKE&t=...
697,unterstützt aber würde eigentlich trotzdem unb...,83,86,unterstützt aber würde eigentlich trotzdem unb...,takzKfwqTOc,https://www.youtube.com/watch?v=takzKfwqTOc&t=83s
698,von diesem ausflug nichts erzählt habe ziemlic...,153,158,von diesem ausflug nichts erzählt habe ziemlic...,f6e3SJdcLL0,https://www.youtube.com/watch?v=f6e3SJdcLL0&t=...


In [11]:
df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = df_10000.head(700)
df_10000

,search_string,start_time,end_time,sentence,video_id,video_url
0,ja schön danke dir ja das kann ich mir auch gu...,4296,4310,ja schön danke dir ja das kann ich mir auch gu...,ffP5IB9bPgQ,https://www.youtube.com/watch?v=ffP5IB9bPgQ&t=...
1,die schon lange nicht mehr benutzt wurde und i...,55,65,wie so eine alte kartoffel die schon lange nic...,5b46anicJdI,https://www.youtube.com/watch?v=5b46anicJdI&t=55s
2,es ist auch eine große erleichterung dass wir ...,727,737,es ist auch eine große erleichterung dass wir ...,gweZAS7SiOU,https://www.youtube.com/watch?v=gweZAS7SiOU&t=...
3,ja die frage wird oft gestellt man macht sich ...,1272,1283,ja die frage wird oft gestellt man macht sich ...,rHKdeRoDJDs,https://www.youtube.com/watch?v=rHKdeRoDJDs&t=...
4,emotionale momente wenn so ein brief bei einer...,5,18,emotionale momente wenn so ein brief bei einer...,prJrgq_RXKg,https://www.youtube.com/watch?v=prJrgq_RXKg&t=5s
...,...,...,...,...,...,...
695,deutschland sicher fühlen können menschen halt...,1060,1068,deutschland sicher fühlen können menschen halt...,_3cEu55RphU,https://www.youtube.com/watch?v=_3cEu55RphU&t=...
696,anders besser vielleicht ausgesehen schreibt e...,414,420,anders besser vielleicht ausgesehen schreibt e...,8Nq1TUbjZtI,https://www.youtube.com/watch?v=8Nq1TUbjZtI&t=...
697,gelände aufgaben erfüllen zum beispiel irgendw...,32,39,gelände aufgaben erfüllen zum beispiel irgendw...,YCV743_ZnvY,https://www.youtube.com/watch?v=YCV743_ZnvY&t=32s
698,kreuz auf jeden fall wieder beide beine vor di...,522,527,kreuz auf jeden fall wieder beide beine vor di...,A3xTBqtbgnA,https://www.youtube.com/watch?v=A3xTBqtbgnA&t=...


In [12]:
df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = df_20000.head(700)
df_20000

,search_string,start_time,end_time,sentence,video_id,video_url
0,so dieses ich hab das jetzt verstanden das mac...,3446,3461,so dieses ich hab das jetzt verstanden das mac...,ffP5IB9bPgQ,https://www.youtube.com/watch?v=ffP5IB9bPgQ&t=...
1,wenn sie das auch selber sehen sie müssten wir...,2328,2340,wenn sie das auch selber sehen sie müssten wir...,tU_GyKFEvsA,https://www.youtube.com/watch?v=tU_GyKFEvsA&t=...
2,wenn ich die konstruktion verstehe das ist ebe...,1468,1481,wenn ich die konstruktion verstehe das ist ebe...,WpcpACck71g,https://www.youtube.com/watch?v=WpcpACck71g&t=...
3,er zu dem zeitpunkt noch ein patient ist dass ...,954,966,er zu dem zeitpunkt noch ein patient ist dass ...,prJrgq_RXKg,https://www.youtube.com/watch?v=prJrgq_RXKg&t=...
4,ja wir freuen uns total mit ihnen dass das gek...,1144,1154,ja wir freuen uns total mit ihnen dass das gek...,rHKdeRoDJDs,https://www.youtube.com/watch?v=rHKdeRoDJDs&t=...
...,...,...,...,...,...,...
695,große ball großen onkel festen stand suchen nä...,2184,2191,zetteln große ball großen onkel festen stand s...,bWiNOoezhUs,https://www.youtube.com/watch?v=bWiNOoezhUs&t=...
696,freunden rüber schicken ganz interessante gesc...,624,629,freunden rüber schicken ganz interessante gesc...,Y2Aoz_OURpM,https://www.youtube.com/watch?v=Y2Aoz_OURpM&t=...
697,damit gibts jetzt keine ausreden mehr denn die...,580,586,damit gibts jetzt keine ausreden mehr denn die...,RjzqJEVSW2o,https://www.youtube.com/watch?v=RjzqJEVSW2o&t=...
698,angesichts dieser ungewöhnlichen umstände neue...,24,30,angesichts dieser ungewöhnlichen umstände neue...,JtCLP8EH3lQ,https://www.youtube.com/watch?v=JtCLP8EH3lQ&t=24s


In [13]:
df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = df_40000.head(700)
df_40000

,search_string,start_time,end_time,sentence,video_id,video_url
0,doch noch bevor ich antworten konnte schrie si...,358,366,doch noch bevor ich antworten konnte schrie si...,9k2_UwkcYWo,https://www.youtube.com/watch?v=9k2_UwkcYWo&t=...
1,so dieses ich hab das jetzt verstanden das mac...,3446,3461,so dieses ich hab das jetzt verstanden das mac...,ffP5IB9bPgQ,https://www.youtube.com/watch?v=ffP5IB9bPgQ&t=...
2,nun noch einmal ganz kurz ich glaube nicht all...,4052,4065,nun noch einmal ganz kurz ich glaube nicht all...,ffP5IB9bPgQ,https://www.youtube.com/watch?v=ffP5IB9bPgQ&t=...
3,die fragen sind nicht so willkürlich wie bei e...,2515,2525,die fragen sind nicht so willkürlich wie bei e...,ffP5IB9bPgQ,https://www.youtube.com/watch?v=ffP5IB9bPgQ&t=...
4,mal zurück zu ihrer tätigkeit als solche mich ...,758,774,mal zurück zu ihrer tätigkeit als solche mich ...,URZkhWJpxs4,https://www.youtube.com/watch?v=URZkhWJpxs4&t=...
...,...,...,...,...,...,...
695,video zum thema augenbrauen richtig schminken ...,119,124,video zum thema augenbrauen richtig schminken ...,l27FkdJfRSk,https://www.youtube.com/watch?v=l27FkdJfRSk&t=...
696,er würde bei einem crash wahrscheinlich weiter...,271,280,er würde bei einem crash wahrscheinlich weiter...,DKYgzLkmvH4,https://www.youtube.com/watch?v=DKYgzLkmvH4&t=...
697,schreibe zum beispiel sehr geehrte frau müller...,152,156,schreibe zum beispiel sehr geehrte frau müller...,A3bpkMGriUo,https://www.youtube.com/watch?v=A3bpkMGriUo&t=...
698,dem projekt wurden studenten nur theoretisch g...,267,274,dem projekt wurden studenten nur theoretisch g...,oPsV3JT8_78,https://www.youtube.com/watch?v=oPsV3JT8_78&t=...


#### End Time Reassign

In [14]:
df_28_reassign = df_28.copy() # important because when it doesn't copy, main df are also change.
df_200_reassign = df_200.copy()
df_1000_reassign = df_1000.copy()
df_5000_reassign = df_5000.copy()
df_10000_reassign = df_10000.copy()
df_20000_reassign = df_20000.copy()
df_40000_reassign = df_40000.copy()

In [15]:
df_28_reassign["end_time"] = df_28_reassign["start_time"] + 1
df_200_reassign["end_time"] = df_200_reassign["start_time"] + 1
df_1000_reassign["end_time"] = df_1000_reassign["start_time"] + 1
df_5000_reassign["end_time"] = df_5000_reassign["start_time"] + 1
df_10000_reassign["end_time"] = df_10000_reassign["start_time"] + 1
df_20000_reassign["end_time"] = df_20000_reassign["start_time"] + 1
df_40000_reassign["end_time"] = df_40000_reassign["start_time"] + 1

In [16]:
out_path = "/home/kurubal/Downloads"

In [17]:
df_28_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx", index=False)
df_200_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx", index=False)
df_1000_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx", index=False)
df_5000_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx", index=False)
df_10000_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx", index=False)
df_20000_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx", index=False)
df_40000_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx", index=False)

In [ ]:
# !!!!! End Time Reassign files result check on youtube tool and change file with youtube tool file result as same name (old file should remove) !!!!! 

#### End Time Reassign Convert Orjinal (From Youtube Tool Result)

In [17]:
df_28_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx")
df_28_video_list = list(set(df_28_youtube_tool["video_id"]))

In [18]:
df_200_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx")
df_200_video_list = list(set(df_200_youtube_tool["video_id"]))

In [19]:
df_1000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx")
df_1000_video_list = list(set(df_1000_youtube_tool["video_id"]))

In [20]:
df_5000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx")
df_5000_video_list = list(set(df_5000_youtube_tool["video_id"]))

In [21]:
df_10000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx")
df_10000_video_list = list(set(df_10000_youtube_tool["video_id"]))

In [22]:
df_20000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx")
df_20000_video_list = list(set(df_20000_youtube_tool["video_id"]))

In [23]:
df_40000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx")
df_40000_video_list = list(set(df_40000_youtube_tool["video_id"]))

##### Video List In video_id 

In [24]:
df_28_new = df_28[df_28["video_id"].isin(df_28_video_list)]
#df_28_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv", index=False)
df_28_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx", index=False)
df_28_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,ist sie aber so aber was ist mit mir,644,648,und dann ist sie aber so aber was ist mit mir,E5wrYo4PLM4,https://www.youtube.com/watch?v=E5wrYo4PLM4&t=...
1,und nicht auf das was du nicht,391,393,und nicht auf das was du nicht hast,D2EWOivrzz8,https://www.youtube.com/watch?v=D2EWOivrzz8&t=...
2,nicht dass das so was ist das,687,690,oh nee sag nicht dass das so was ist das,m9hZzWA0R3k,https://www.youtube.com/watch?v=m9hZzWA0R3k&t=...
3,das ist ein dass das nicht so,578,580,das ist ein dass das nicht so,XGhAlBQ3gOY,https://www.youtube.com/watch?v=XGhAlBQ3gOY&t=...
4,das ist ja nicht das was ich,548,551,das ist ja nicht das was ich,X0g4THI7vEs,https://www.youtube.com/watch?v=X0g4THI7vEs&t=...
...,...,...,...,...,...,...
302,ja aber ich,304,307,ja aber ich,Fhj19V34vzU,https://www.youtube.com/watch?v=Fhj19V34vzU&t=...
303,und mit den,436,438,und mit den,xNbTBGajGdQ,https://www.youtube.com/watch?v=xNbTBGajGdQ&t=...
304,dass er mir,3880,3884,dass er mir,PwbNezj_pyw,https://www.youtube.com/watch?v=PwbNezj_pyw&t=...
305,dass in den,602,608,dass in den,CSOH6GQhBQ4,https://www.youtube.com/watch?v=CSOH6GQhBQ4&t=...


In [25]:
df_200_new = df_200[df_200["video_id"].isin(df_200_video_list)]
#df_200_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv", index=False)
df_200_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx", index=False)
df_200_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,oder da würde er sagen da ich glaube es geht l...,1499,1505,oder da würde er sagen da ich glaube es geht l...,L22SNtHYLCw,https://www.youtube.com/watch?v=L22SNtHYLCw&t=...
1,sehen kann dann machen wir alles zu genau also...,2117,2126,sehen kann dann machen wir alles zu genau also...,8BgkmfGO8SI,https://www.youtube.com/watch?v=8BgkmfGO8SI&t=...
2,wie es ihnen gehen muss und was jetzt los ist ...,408,414,wie es ihnen gehen muss und was jetzt los ist ...,3sTLcfBgJh4,https://www.youtube.com/watch?v=3sTLcfBgJh4&t=...
3,ist etwas nicht so wirklich ist und dann mache...,2745,2754,ist etwas nicht so wirklich ist und dann mache...,HWzDSgiLVkI,https://www.youtube.com/watch?v=HWzDSgiLVkI&t=...
4,du denn jetzt machen oder sagen ja was können ...,2046,2051,du denn jetzt machen oder sagen ja was können ...,L22SNtHYLCw,https://www.youtube.com/watch?v=L22SNtHYLCw&t=...
...,...,...,...,...,...,...
695,ihn doch nur für sein geld,282,284,du liebst ihn doch nur für sein geld,yr9WrUFM4IE,https://www.youtube.com/watch?v=yr9WrUFM4IE&t=...
696,einen oder zwei leute raus,277,279,einen oder zwei leute raus,Ut64cNI2kvg,https://www.youtube.com/watch?v=Ut64cNI2kvg&t=...
697,wer hat schon zeit für den,607,610,wer hat schon zeit für den anderen,O5unu7USjv8,https://www.youtube.com/watch?v=O5unu7USjv8&t=...
698,nein wirklich oh mein gott,617,621,nein wirklich oh mein gott,YwTXhNYG4gw,https://www.youtube.com/watch?v=YwTXhNYG4gw&t=...


In [26]:
df_1000_new = df_1000[df_1000["video_id"].isin(df_1000_video_list)]
#df_1000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv", index=False)
df_1000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx", index=False)
df_1000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,der noch geld haben will wahrscheinlich eben d...,878,884,der noch geld haben will wahrscheinlich eben d...,e0PyOh2KMQw,https://www.youtube.com/watch?v=e0PyOh2KMQw&t=...
1,aber vielleicht leute zu haben die da sind wo ...,815,824,aber vielleicht leute zu haben die da sind wo ...,e0PyOh2KMQw,https://www.youtube.com/watch?v=e0PyOh2KMQw&t=...
2,ich glaube sie wollte sagen hey du bist nur ei...,3278,3285,ich glaube sie wollte sagen hey du bist nur ei...,twfbyHqyITc,https://www.youtube.com/watch?v=twfbyHqyITc&t=...
3,so stelle ich mir das jetzt vor weil wenn man ...,1803,1810,so stelle ich mir das jetzt vor weil wenn man ...,URWnSf1sEDI,https://www.youtube.com/watch?v=URWnSf1sEDI&t=...
4,das hat damit zu tun dass wir einfach wissen d...,2804,2809,das hat damit zu tun dass wir einfach wissen d...,V1rKsO0okzY,https://www.youtube.com/watch?v=V1rKsO0okzY&t=...
...,...,...,...,...,...,...
695,werden weil wenn von uns da jemand sitzen würde,1025,1030,einbezogen werden weil wenn von uns da jemand ...,bXt7do8CjxU,https://www.youtube.com/watch?v=bXt7do8CjxU&t=...
696,am nächsten tag machte er sich auf den weg nach,1404,1407,am nächsten tag machte er sich auf den weg nac...,kLZyfnwuFAk,https://www.youtube.com/watch?v=kLZyfnwuFAk&t=...
697,am abend vorher meine sachen hier auf den boden,1608,1611,ich warf am abend vorher meine sachen hier auf...,XLxx_oJVkiM,https://www.youtube.com/watch?v=XLxx_oJVkiM&t=...
698,am nächsten tag machte er sich auf den weg nach,605,609,am nächsten tag machte er sich auf den weg nac...,1uzFP9Y9gZk,https://www.youtube.com/watch?v=1uzFP9Y9gZk&t=...


In [27]:
df_5000_new = df_5000[df_5000["video_id"].isin(df_5000_video_list)]
#df_5000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv", index=False)
df_5000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx", index=False)
df_5000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,lassen wir haben nachher sehr oft da drüber ge...,608,618,ja meine frau hat sich das nie so anmerken las...,rHKdeRoDJDs,https://www.youtube.com/watch?v=rHKdeRoDJDs&t=...
1,einige haben ja ganz schön viel erlebt ich den...,366,381,einige haben ja ganz schön viel erlebt ich den...,_W671aWBZL4,https://www.youtube.com/watch?v=_W671aWBZL4&t=...
2,davon glaube ich geht man so stück für stück w...,1727,1736,davon glaube ich geht man so stück für stück w...,kg23NfjKeTc,https://www.youtube.com/watch?v=kg23NfjKeTc&t=...
3,eine gewisse sicherheit bekommen dass das was ...,135,147,eine gewisse sicherheit bekommen dass das was ...,gweZAS7SiOU,https://www.youtube.com/watch?v=gweZAS7SiOU&t=...
5,und normalerweise bricht meine haut ein paar t...,447,458,ich habe gerade meine periode und normalerweis...,niPfa1UCmaE,https://www.youtube.com/watch?v=niPfa1UCmaE&t=...
...,...,...,...,...,...,...
695,machen willst dein leben lang machen wolltest ...,318,322,machen willst dein leben lang machen wolltest ...,X2kuDag0LA4,https://www.youtube.com/watch?v=X2kuDag0LA4&t=...
696,wahrscheinlich wollte er sich von seinem alten...,3251,3256,wahrscheinlich wollte er sich von seinem alten...,SsNCFbkXGKE,https://www.youtube.com/watch?v=SsNCFbkXGKE&t=...
697,unterstützt aber würde eigentlich trotzdem unb...,83,86,unterstützt aber würde eigentlich trotzdem unb...,takzKfwqTOc,https://www.youtube.com/watch?v=takzKfwqTOc&t=83s
698,von diesem ausflug nichts erzählt habe ziemlic...,153,158,von diesem ausflug nichts erzählt habe ziemlic...,f6e3SJdcLL0,https://www.youtube.com/watch?v=f6e3SJdcLL0&t=...


In [28]:
df_10000_new = df_10000[df_10000["video_id"].isin(df_10000_video_list)]
#df_10000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv", index=False)
df_10000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx", index=False)
df_10000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,ja schön danke dir ja das kann ich mir auch gu...,4296,4310,ja schön danke dir ja das kann ich mir auch gu...,ffP5IB9bPgQ,https://www.youtube.com/watch?v=ffP5IB9bPgQ&t=...
1,die schon lange nicht mehr benutzt wurde und i...,55,65,wie so eine alte kartoffel die schon lange nic...,5b46anicJdI,https://www.youtube.com/watch?v=5b46anicJdI&t=55s
2,es ist auch eine große erleichterung dass wir ...,727,737,es ist auch eine große erleichterung dass wir ...,gweZAS7SiOU,https://www.youtube.com/watch?v=gweZAS7SiOU&t=...
3,ja die frage wird oft gestellt man macht sich ...,1272,1283,ja die frage wird oft gestellt man macht sich ...,rHKdeRoDJDs,https://www.youtube.com/watch?v=rHKdeRoDJDs&t=...
4,emotionale momente wenn so ein brief bei einer...,5,18,emotionale momente wenn so ein brief bei einer...,prJrgq_RXKg,https://www.youtube.com/watch?v=prJrgq_RXKg&t=5s
...,...,...,...,...,...,...
695,deutschland sicher fühlen können menschen halt...,1060,1068,deutschland sicher fühlen können menschen halt...,_3cEu55RphU,https://www.youtube.com/watch?v=_3cEu55RphU&t=...
696,anders besser vielleicht ausgesehen schreibt e...,414,420,anders besser vielleicht ausgesehen schreibt e...,8Nq1TUbjZtI,https://www.youtube.com/watch?v=8Nq1TUbjZtI&t=...
697,gelände aufgaben erfüllen zum beispiel irgendw...,32,39,gelände aufgaben erfüllen zum beispiel irgendw...,YCV743_ZnvY,https://www.youtube.com/watch?v=YCV743_ZnvY&t=32s
698,kreuz auf jeden fall wieder beide beine vor di...,522,527,kreuz auf jeden fall wieder beide beine vor di...,A3xTBqtbgnA,https://www.youtube.com/watch?v=A3xTBqtbgnA&t=...


In [29]:
df_20000_new = df_20000[df_20000["video_id"].isin(df_20000_video_list)]
#df_20000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv", index=False)
df_20000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx", index=False)
df_20000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,so dieses ich hab das jetzt verstanden das mac...,3446,3461,so dieses ich hab das jetzt verstanden das mac...,ffP5IB9bPgQ,https://www.youtube.com/watch?v=ffP5IB9bPgQ&t=...
1,wenn sie das auch selber sehen sie müssten wir...,2328,2340,wenn sie das auch selber sehen sie müssten wir...,tU_GyKFEvsA,https://www.youtube.com/watch?v=tU_GyKFEvsA&t=...
2,wenn ich die konstruktion verstehe das ist ebe...,1468,1481,wenn ich die konstruktion verstehe das ist ebe...,WpcpACck71g,https://www.youtube.com/watch?v=WpcpACck71g&t=...
3,er zu dem zeitpunkt noch ein patient ist dass ...,954,966,er zu dem zeitpunkt noch ein patient ist dass ...,prJrgq_RXKg,https://www.youtube.com/watch?v=prJrgq_RXKg&t=...
4,ja wir freuen uns total mit ihnen dass das gek...,1144,1154,ja wir freuen uns total mit ihnen dass das gek...,rHKdeRoDJDs,https://www.youtube.com/watch?v=rHKdeRoDJDs&t=...
...,...,...,...,...,...,...
695,große ball großen onkel festen stand suchen nä...,2184,2191,zetteln große ball großen onkel festen stand s...,bWiNOoezhUs,https://www.youtube.com/watch?v=bWiNOoezhUs&t=...
696,freunden rüber schicken ganz interessante gesc...,624,629,freunden rüber schicken ganz interessante gesc...,Y2Aoz_OURpM,https://www.youtube.com/watch?v=Y2Aoz_OURpM&t=...
697,damit gibts jetzt keine ausreden mehr denn die...,580,586,damit gibts jetzt keine ausreden mehr denn die...,RjzqJEVSW2o,https://www.youtube.com/watch?v=RjzqJEVSW2o&t=...
698,angesichts dieser ungewöhnlichen umstände neue...,24,30,angesichts dieser ungewöhnlichen umstände neue...,JtCLP8EH3lQ,https://www.youtube.com/watch?v=JtCLP8EH3lQ&t=24s


In [30]:
df_40000_new = df_40000[df_40000["video_id"].isin(df_40000_video_list)]
#df_40000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv", index=False)
df_40000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx", index=False)
df_40000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,doch noch bevor ich antworten konnte schrie si...,358,366,doch noch bevor ich antworten konnte schrie si...,9k2_UwkcYWo,https://www.youtube.com/watch?v=9k2_UwkcYWo&t=...
1,so dieses ich hab das jetzt verstanden das mac...,3446,3461,so dieses ich hab das jetzt verstanden das mac...,ffP5IB9bPgQ,https://www.youtube.com/watch?v=ffP5IB9bPgQ&t=...
2,nun noch einmal ganz kurz ich glaube nicht all...,4052,4065,nun noch einmal ganz kurz ich glaube nicht all...,ffP5IB9bPgQ,https://www.youtube.com/watch?v=ffP5IB9bPgQ&t=...
3,die fragen sind nicht so willkürlich wie bei e...,2515,2525,die fragen sind nicht so willkürlich wie bei e...,ffP5IB9bPgQ,https://www.youtube.com/watch?v=ffP5IB9bPgQ&t=...
4,mal zurück zu ihrer tätigkeit als solche mich ...,758,774,mal zurück zu ihrer tätigkeit als solche mich ...,URZkhWJpxs4,https://www.youtube.com/watch?v=URZkhWJpxs4&t=...
...,...,...,...,...,...,...
695,video zum thema augenbrauen richtig schminken ...,119,124,video zum thema augenbrauen richtig schminken ...,l27FkdJfRSk,https://www.youtube.com/watch?v=l27FkdJfRSk&t=...
696,er würde bei einem crash wahrscheinlich weiter...,271,280,er würde bei einem crash wahrscheinlich weiter...,DKYgzLkmvH4,https://www.youtube.com/watch?v=DKYgzLkmvH4&t=...
697,schreibe zum beispiel sehr geehrte frau müller...,152,156,schreibe zum beispiel sehr geehrte frau müller...,A3bpkMGriUo,https://www.youtube.com/watch?v=A3bpkMGriUo&t=...
698,dem projekt wurden studenten nur theoretisch g...,267,274,dem projekt wurden studenten nur theoretisch g...,oPsV3JT8_78,https://www.youtube.com/watch?v=oPsV3JT8_78&t=...


#### Copy Move And Delete

In [31]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_*_Word{file_ext}.*")
output_file2

['German_Word_Group_Max_In_Youtube_Sentence_28_Word6.xlsx',
 'German_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx',
 'German_Word_Group_Max_In_Youtube_Sentence_1000_Word6.xlsx',
 'German_Word_Group_Max_In_Youtube_Sentence_5000_Word6.xlsx',
 'German_Word_Group_Max_In_Youtube_Sentence_10000_Word6.xlsx',
 'German_Word_Group_Max_In_Youtube_Sentence_20000_Word6.xlsx',
 'German_Word_Group_Max_In_Youtube_Sentence_40000_Word6.xlsx']

In [32]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [33]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass